**TODO**
1. gdf can only contain 1 type of datasetName (like "LANDSAT_8_C1"). It's hard to get the dataset name from older datasets (like CORONA)
2. updateOrderScene only takes a gdf as an argument; allow this to handle lists and string
3. sceneSearch only works for points; allow to handle areas
4. clearOrder should print a simpler message
5. datasetSearch is basically the same as sceneSearch, but with a different action word and a different return

In [1]:
import json
import time
#import re

import requests as rq
import pandas as pd
from pandas.io.json import json_normalize
import geopandas as gpd
from shapely.geometry import Polygon

from ee_secret_codes import *

endpoint = "https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/"

# Login parameters
username = EE_USERNAME
password = EE_PASSWORD

In [2]:
def create_request(request_dict):
    
    """
    Wraps payload dictionary in correct format to pass as a request.
    
    Parameters
    ----------
    request_dict: dictionary or dict-like object
    
    Returns
    -------
    json-like object; dictionary
    
    """
    
    return {'jsonRequest': json.dumps(request_dict)}

In [3]:
def api_call(action, payload=None):
    
    """
    Call EarthExplorer API using endpoint word,different for each
    type of call.
    
    Parameters
    ----------
    action: string
        Keyword specifying endpoint, such as "search"
        
    payload: dict
        Containing parameters required for successful API
        call. At minimum an api key
        
    Returns
    -------
    return_json: python JSON-like object
        JSON object returned from request call
    
    """
    
    if payload:
        params = create_request(payload)

        return_json = rq.get((endpoint + action), params=params).json()

    else:

        return_json = rq.get((endpoint + action)).json()

    return return_json

In [80]:
def login(username, password):
    
    """
    Takes in username and password to generate an API key,
    required for all API calls. Does not use api_call since
    this is a POST request. Set api_key as a global variable.
    
    Parameters
    ----------
    username: string
    
    password: string
    
    Returns
    -------
    api key: string
    
    """

    # Set up auths dict
    login_payload = {'username': username,
                     'password': password,
                     'catalogID': "EE"}

    login_params = create_request(login_payload)

    # Post auths
    login_response = rq.post((endpoint + "login?"), data=login_params).json()

    # Return API key
    return login_response['data']

api_key = login(username, password)

In [5]:
def logout():
    
    """
    Logs user out of system
    
    Parameters
    ----------
    none
    
    Returns
    -------
    logout_response: JSON
        With details of successful logout
    
    """
    
    logout_payload = {"apiKey": api_key}
                      
    logout_response = api_call("logout", logout_payload)
    
    return logout_response

In [6]:
def status():
    """
    Checks status of current user: logged in or not logged in
    """
    status_params = create_request(api_key) 
    
    status_response = api_call("status")
    
    return status_response

In [7]:
def clearOrder():
    
    """Clears item basket and returns response
    
    Parameters
    ----------
    none
    
    Returns
    -------
    clear_response: JSON
        General info on basket contents
        
    """
    clear_payload = {"apiKey": api_key}

    clear_response = api_call("clearorder", clear_payload)

    return clear_response

In [8]:
def itemBasket():
    
    """
    Returns contents of item basket as JSON
    
    Parameters
    ----------
    none
    
    Returns
    -------
    basket: JSON
        Info on scenes added to basket
        
    """
    
    basket_payload = {"apiKey": api_key}
    
    basket = api_call("itembasket", basket_payload)
    
    return basket

In [87]:
def sceneSearch(dataset_name, start_date, end_date,
                latitude=None, longitude=None,
                max_results=None, return_ftype='gdf'):
    """
    Searches scenes from specified dataset. Only works for
    points, not areas.

    Parameters
    ----------
    dataset_name: string

    start_date: string 
        Format YYYY-MM-DD

    latitude: int or float
        In decimal degrees, WGS84

    longitude: as above

    max_results: int (optional)

    return_gdf: boolean
        True to return geodataframe,
        False to return JSON

    Returns
    -------
    search_resp: geodataframe or JSON
    """

    # Create empty dict
    search_payload = {"datasetName": dataset_name,
                      "temporalFilter": {"startDate": start_date,
                                         "endDate": end_date}}

    # If latitude and longitude
    if latitude and longitude:
        search_payload['spatialFilter'] = {"filterType": "mbr",
                                           "lowerLeft": {"latitude": latitude,
                                                         "longitude": longitude},
                                           "upperRight": {"latitude": latitude,
                                                          "longitude": longitude}}
    if max_results:
        search_payload['maxResults'] = max_results

    # Add api key
    search_payload['apiKey'] = api_key

    # Get results as json
    search_resp = api_call("search", search_payload)

    if return_ftype == 'json':
        
        return search_resp

    else:

        return get_gdf(search_resp)

In [35]:
def get_gdf(search_resp_json):
    
    """
    Converts a search response JSON to a geodataframe
    
    Parameters
    ----------
    search_resp_json: dict
        JSON-like object returned from a GET request
        
    Returns
    -------
    gdf: geoDataFrame
    
    """
    
    # normalize json to create df
    df = json_normalize(search_resp_json['data']['results'])

    # Create list to hold Polygon object from each row
    rows_list = []

    # Loop through each scene item in json
    for scene in search_resp_json['data']['results']:

        # DataFrame holds Polygons as a nested list, need to
        # change format to [(x1, y1), (x2, y2)] for shapely
        # Start with "packed list" in df
        packed_list = scene.get('spatialFootprint').get('coordinates')[0]

        # Change each item to a tuple and create a simple list
        unpacked_list = [tuple(sublist) for sublist in packed_list]

        # Create Polygon object for each lit of tuples
        rows_list.append(Polygon(unpacked_list))

    # Create new column and pass it the list
    # of Polygon objects, one for each row
    df['geometry'] = rows_list

    # Create gdf from dataframe, pass in CRS
    gdf = gpd.GeoDataFrame(df, crs='epsg:4326')

    """# Drop columns
    try:
        gdf.drop(columns=['bulkOrdered', 'ordered', 'summary', 'spatialFootprint.coordinates'],
                 inplace=True)
    except:
        pass"""
    try:
        # Get list of fields to convert to datetime object
        date_fields = ['acquisitionDate', "startTime", "endTime"]

        # For each date field, convert to datetime format
        for date_field in date_fields:
            gdf[date_field] = pd.to_datetime(gdf[date_field], 
                                             infer_datetime_format=True)

        #pd.set_option('display.max_colwidth', -1)

        return gdf
    
    except KeyError:
        
        print("Unable to parse date fields")
        
        return gdf

In [11]:
def orderProducts(dataset_name, gdf):
    
    """
    Given a dataset name and geodataframe, adds gdf items to basket
    
    Parameters
    ----------
    dataset_name: string
    
    gdf: geodataframe
        Containing scenes of interest
        
    Returns
    -------
    (none)
    
    """
    
    # Convert entity id column to list
    entity_id_list = gdf.entityId.tolist()
    
    # Set up payload
    op_payload = {"datasetName": dataset_name,
                  "apiKey": api_key,
                  "entityIds": entity_id_list}
    
    # Get JSON of available products; opr = order_products_response
    opr = api_call("getorderproducts", op_payload)

    # Create empty dict to hold 'productId': 'productCode'
    order_dict = {}

    # Loop thru json
    for entity in range(0, len(opr['data'])):

        # Extract entityId
        entity_id = opr['data'][entity].get('entityId')

        # Extract product code
        product_code = opr['data'][0].get('availableProducts')[
            0].get('productCode')

        order_dict[entity_id] = product_code

    updateOrderScene(dataset_name, order_dict)
    
    return

In [12]:
def updateOrderScene(dataset_name, order_dict):
    
    """
    Adds order scene to basket by getting entityId from a dict
    
    Parameters
    ----------
    dataset_name: string
    
    order_dict: dictionary
        Of format{entityId: productCode}
        
    Returns
    -------
    none
    
    """
    
    for entity_id, product_code in order_dict.items():

        order_scene_payload = {"apiKey": api_key,
                               "datasetName": dataset_name,
                               "entityId": entity_id,
                               "productCode": product_code,
                               "option": "None",
                               "outputMedia": "DWNLD"}

        api_call("updateorderscene",
                 order_scene_payload)

    return

In [62]:
def submitOrder():
    
    """Submits current item basket
    
    Parameters
    ------
    none
    
    Returns
    -------
    order_resp: JSON
        Returned after order placed
    
    """

    order_payload = {"apiKey": api_key}

    # Allow user to cancel
    ready = str(input("Ready to submit basket? (Y/N)"))
    
    # Sure, why not
    if ready.lower() in ('y', 'yes', 'ok', '1').:

        order_resp = api_call("submitorder", order_payload)

        return order_resp

    else:
        
        print("Order cancelled")

In [14]:
def datasetSearch(dataset_name=None,
                  start_date=None, end_date=None,
                  latitude=None, longitude=None):
    """Searches all available datasets"""
    dataset_search_payload = {"apiKey": api_key}

    if dataset_name:
        dataset_search_payload["datasetName"] = dataset_name

    if latitude and longitude:
        dataset_search_payload['spatialFilter'] = {"filterType": "mbr",
                                                   "lowerLeft": {"latitude": latitude,
                                                                 "longitude": longitude},
                                                   "upperRight": {"latitude": latitude,
                                                                  "longitude": longitude}
                                                   }

    if start_date and end_date:
        dataset_search_payload['temporalFilter'] = {"startDate": start_date,
                                                    "endDate": end_date}

    ds_search_response = api_call("datasets", dataset_search_payload)

    return ds_search_response